# Arachne (Pareto) で特定された重み数の平均を取得

In [3]:
import os
import numpy as np
from itertools import product
from utils.constant import ViTExperiment, Experiment1, ExperimentRepair1, ExperimentRepair2, Experiment4


ds = "tiny_imagenet"
# k_list = range(5)
k_list = [0]
tgt_rank_list = range(1, 4)
# tgt_rank_list = range(1, 4)
misclf_type_list = ["src_tgt", "tgt"]
# misclf_type_list = ["all", "src_tgt", "tgt"]
fpfn_list = [None, "fp", "fn"]
results = []
num_pos_list = []
for k, tgt_rank, misclf_type, fpfn in product(k_list, tgt_rank_list, misclf_type_list, fpfn_list):
    if (misclf_type == "src_tgt" or misclf_type == "all") and fpfn is not None: # misclf_type == "src_tgt" or "all"の時はfpfnはNoneだけでいい
        # print(f"Skipping: misclf_type={misclf_type} with fpfn={fpfn} is not valid.")
        continue
    if misclf_type == "tgt" and fpfn is None:
        # print(f"Skipping: misclf_type={misclf_type} with fpfn={fpfn} is not valid.")
        continue
    pretrained_dir = getattr(ViTExperiment, ds).OUTPUT_DIR.format(k=k)
    if fpfn is not None and misclf_type == "tgt":
        location_save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_{fpfn}_weights_location")
    elif misclf_type == "all":
        location_save_dir = os.path.join(pretrained_dir, f"all_weights_location")
    else:
        location_save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_weights_location")
    # old_location_save_path = os.path.join(location_save_dir, f"exp-fl-2_location_weight_bl.npy")
    location_save_path = os.path.join(location_save_dir, f"exp-fl-2_location_pareto_weight_bl.npy")
    pos_before, pos_after = np.load(location_save_path, allow_pickle=True)
    num_pos = len(pos_before) + len(pos_after)
    num_pos_list.append(num_pos)
    print(f"ds={ds}, k={k}, tgt_rank={tgt_rank}, misclf_type={misclf_type}, fpfn={fpfn}, num_pos={num_pos}")

ds=tiny_imagenet, k=0, tgt_rank=1, misclf_type=src_tgt, fpfn=None, num_pos=11
ds=tiny_imagenet, k=0, tgt_rank=1, misclf_type=tgt, fpfn=fp, num_pos=18
ds=tiny_imagenet, k=0, tgt_rank=1, misclf_type=tgt, fpfn=fn, num_pos=10
ds=tiny_imagenet, k=0, tgt_rank=2, misclf_type=src_tgt, fpfn=None, num_pos=10
ds=tiny_imagenet, k=0, tgt_rank=2, misclf_type=tgt, fpfn=fp, num_pos=7
ds=tiny_imagenet, k=0, tgt_rank=2, misclf_type=tgt, fpfn=fn, num_pos=12
ds=tiny_imagenet, k=0, tgt_rank=3, misclf_type=src_tgt, fpfn=None, num_pos=12
ds=tiny_imagenet, k=0, tgt_rank=3, misclf_type=tgt, fpfn=fp, num_pos=8
ds=tiny_imagenet, k=0, tgt_rank=3, misclf_type=tgt, fpfn=fn, num_pos=11


In [4]:
# 平均の重み数を表示
print(f"Average number of positive weights: {np.mean(num_pos_list)}")

Average number of positive weights: 11.0


# 結果
パレートフロントを取ると平均(11個, c100), (11個, tiny-imagenet) くらい重みが特定できる．

Arachneの論文では以下のようにあったのでそんなに違ってない．
>Since the average number of localised neural weights is 11.6, 7.6 and 14.3 respectively for C10, FM and GTSRB, GL and RS localisation methods were configured to select 12, 8 and 14 weights for C10, FM and GTSRB, respectively.

ただし，ネットワークの規模が違うので，この数の重みではきっと不十分だと思う．

従来手法のArachneのようなパレートフロントを取るだけでは大規模なネットワークに対応できない，という点は押しても良いかも．論文のmotivationとして．